# Testing felt-python

In [ ]:
import os

from felt_python import (
    create_map,
    delete_map,
    get_map_details,
    list_layers,
    upload_file,
    upload_url,
    upload_dataframe,
    refresh_file_layer,
    refresh_url_layer,
    get_layer_style,
    update_layer_style
)

os.environ["FELT_API_TOKEN"] = "felt_pat_wZOSz45T+BoT89uRzfeu3eRQkUj9iQxvXplMSU8Erpo"

# Maps

Create a map and retrieve its details, storing the ID in a variable

In [ ]:
resp = create_map(
    title="A felt-py map",
    lat=40,
    lon=-3,
    public_access="private",
)
map_id = resp["id"]

In [ ]:
get_map_details(map_id)

# Layers

Create layers from file, URL and (Geo)DataFrame uploads

In [ ]:
map_id

## Upload a file

In [ ]:
layer_resp = upload_file(map_id, "tests/fixtures/null-island-points-sample.geojson", "The Points Layer")

In [ ]:
# Ideally, the new layer ID would be present here
layer_resp.content

In [ ]:
# Let's fetch it
layers = list_layers(map_id)

In [ ]:
layer_id = layers[0]["id"]

In [ ]:
layer_id

## Attempt to refresh file upload

**Note**: Refreshing here fails because `list_layers` and `url_import` return layer group IDs but the v1 refresh endpoints need _layer_ IDs. Styling works with both layer IDs _and_ layer group IDs which explains the mismatch

In [ ]:
# Refresh not working, always says layer_id not found
# refresh_file_layer(map_id, layer_id, file_name="tests/fixtures/null-island-points.geojson")

## Upload a URL

In [ ]:
live_earthquakes_url = "https://earthquake.usgs.gov/earthquakes/feed/v1.0/summary/all_hour.geojson"
url_upload = upload_url(map_id, live_earthquakes_url, "Live Earthquakes")
url_upload_id = url_upload["relationships"]["datasets"]["data"][0]["id"]

In [ ]:
url_upload

Refreshing works this time because we've fetched the layer ID instead of the layer group ID

In [ ]:
refresh_url_layer(map_id, url_upload_id)

# Styling

Fetch style of file upload layer

In [ ]:
current_style = get_layer_style(map_id, layer_id)
current_style

Update style

In [ ]:
new_style = current_style.copy()
new_style["style"]["color"] = "red"
new_style["style"]["size"] = 20
update_layer_style(map_id, layer_id, new_style)

# (Geo)DataFrames

In [ ]:
import geopandas as gpd

gdf = gpd.read_file('tests/fixtures/null-island-points.geojson')

In [ ]:
upload_dataframe(map_id, gdf, layer_name="GeoPandas FTW")

# Cleanup

In [ ]:
delete_map(map_id)